In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import os
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier

# Training dataset preprocessing

In [2]:
root = "C:/Users/verdi/Documents/Datasets/IEEE-CIS_Fraud_Detection"
export_path = "C:/Users/verdi/Documents/Datasets/IEEE-CIS_Fraud_Detection/processed"
os.listdir(root + "/ieee-fraud-detection")

['sample_submission.csv',
 'test_identity.csv',
 'test_transaction.csv',
 'train_identity.csv',
 'train_transaction.csv']

In [3]:
# Load both training and identity tables
train_trans_df = pd.read_csv(root + "/ieee-fraud-detection/train_transaction.csv", index_col='TransactionID')
train_id_df = pd.read_csv(root + "/ieee-fraud-detection/train_identity.csv", index_col='TransactionID')

In [4]:
# Merge both tables into a single table
df_train = train_trans_df.merge(train_id_df, how='left', left_index=True, right_index=True)
df_train.to_csv(root + "/processed/merged/train_set.csv")
del train_trans_df
del train_id_df

In [5]:
print("Dataset has {} columns".format(len(df_train.columns)))
df_train.head(5)

Dataset has 433 columns


,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [6]:
# Columns we want to keep without any preprocessing
cols_to_ignore = ['TransactionID']
# Anomaly column
ad_label = 'isFraud'
# Column used for submission
submission_col = 'TransactionID'

# Columns to drop because we do not want to use them during training
#   - TransactionDT: We do not want to discriminate on time
#   - card4: While certain vendors might be subject to more fraud we do not want the model to discriminate based on the card which could lead to false negatives if a frauder decides to change card
cols_to_drop = ['TransactionDT', 'card4']

## Missing values

In [7]:
# Check for missing values
cols_missing_vals = df_train.columns[df_train.isna().sum() >= 1]
print("There are {} columns with missing values".format(len(cols_missing_vals)))

# Threshold beyond which we drop features
thresh = .10 * len(df_train)
nan_to_drop = df_train.columns[df_train.isna().sum() >= thresh]
cols_to_drop.extend(nan_to_drop.tolist())
print("Found {} columns with missing value ratio equal or greater than {:2.2f}".format(len(nan_to_drop), thresh))

# Keep the remaining columns with missing values aside
remaining_nan_cols = list(set(cols_missing_vals) - set(nan_to_drop))
print("Remaining {} columns have to be treated".format(len(remaining_nan_cols)))

# Display all the columns with missing values
df_train[cols_missing_vals].isna().sum().sort_values(ascending=False)

There are 414 columns with missing values
Found 322 columns with missing value ratio equal or greater than 59054.00
Remaining 92 columns have to be treated


id_24    585793
id_25    585408
id_07    585385
id_08    585385
id_21    585381
          ...  
V285         12
V284         12
V280         12
V279         12
V312         12
Length: 414, dtype: int64

In [8]:
# Separate the categorical from the numerical values
num_cols_nan_vals = df_train[remaining_nan_cols].select_dtypes(exclude=['object', 'category']).columns
cat_cols_nan_vals = df_train[remaining_nan_cols].select_dtypes(include=['object', 'category']).columns

# Add the only categorical column with missing values to the drop list
cols_to_drop.extend(cat_cols_nan_vals)
print(num_cols_nan_vals)
print(cat_cols_nan_vals)
df_train[num_cols_nan_vals].describe()

Index(['V119', 'V314', 'V133', 'V290', 'V100', 'V137', 'D1', 'V280', 'V117',
       'card5', 'V134', 'V126', 'V298', 'V289', 'V292', 'V286', 'V295', 'V287',
       'V303', 'V115', 'card2', 'V125', 'V306', 'V288', 'V131', 'V113', 'V136',
       'V297', 'V307', 'V299', 'V321', 'V103', 'V106', 'V98', 'V110', 'V112',
       'V135', 'V316', 'V281', 'V308', 'V296', 'V127', 'V118', 'V99', 'V123',
       'V108', 'V116', 'V107', 'V284', 'V293', 'V312', 'V102', 'V121', 'V309',
       'V300', 'V320', 'V302', 'V313', 'V111', 'V129', 'V104', 'V132', 'V315',
       'V124', 'V130', 'V318', 'V291', 'V105', 'V96', 'V122', 'V283', 'V304',
       'V310', 'V305', 'V285', 'V282', 'V319', 'V109', 'V279', 'V97', 'V101',
       'V311', 'V294', 'V128', 'V317', 'V120', 'card3', 'V95', 'V301', 'V114'],
      dtype='object')
Index(['card6', 'card4'], dtype='object')


,V119,V314,V133,V290,V100,V137,D1,V280,V117,card5,...,V101,V311,V294,V128,V317,V120,card3,V95,V301,V114
count,590226.000000,589271.000000,590226.000000,590528.000000,590226.000000,590226.000000,589271.000000,590528.000000,590226.000000,586281.000000,...,590226.000000,590528.000000,590528.000000,590226.000000,590528.000000,590226.000000,588975.000000,590226.000000,589271.000000,590226.000000
mean,1.000729,43.319174,204.889160,1.103011,0.273504,26.365090,94.347568,1.967082,1.000391,199.278897,...,0.889249,4.202175,2.313863,204.094038,247.606741,1.000874,153.194925,1.038019,0.052002,1.009298
std,0.036392,173.619028,3796.316755,0.768897,0.947176,348.332714,157.660387,27.851780,0.035238,41.244453,...,20.582571,102.374938,39.526468,3010.258774,3980.042828,0.041684,11.336444,21.034304,0.318310,0.110179
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,166.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,150.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,3.000000,0.000000,1.000000,226.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,150.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,122.000000,1.000000,1.000000,226.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,150.000000,0.000000,0.000000,1.000000
max,3.000000,7519.870117,133915.000000,67.000000,28.000000,90750.000000,640.000000,975.000000,3.000000,237.000000,...,869.000000,55125.000000,1286.000000,160000.000000,134021.000000,3.000000,231.000000,880.000000,13.000000,6.000000


In [9]:
# Use SKLearn's SimpleImputer to replace NaN values
# Using the mean is ideal for columns with low variance
# However for columns with high variance, another statistic should be used
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df_train[num_cols_nan_vals] = imputer.fit_transform(df_train[num_cols_nan_vals].values.astype(np.float64))

In [10]:
# Validate the previous step
assert df_train[num_cols_nan_vals].isna().any().sum() == 0
print("{} columns to be dropped".format(len(cols_to_drop)))
# Trick to remove duplicates
cols_to_drop = list(set(cols_to_drop))

326 columns to be dropped


In [11]:
# Drop the unwanted columns
df_train = df_train.drop(columns=cols_to_drop)
df_train.head(5)

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card5,C1,C2,C3,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,68.5,W,13926,362.555488,150.0,142.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0
2987001,0,29.0,W,2755,404.000000,150.0,102.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2987002,0,59.0,W,4663,490.000000,150.0,166.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2987003,0,50.0,W,18132,567.000000,150.0,117.0,2.0,5.0,0.0,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0
2987004,0,50.0,H,4497,514.000000,150.0,102.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Unique values

In [12]:
# Check for columns with unique values
uniq_cols = df_train.columns[df_train.nunique() == 1]
print("Dropping {} columns with unique values".format(len(uniq_cols)))
if len(uniq_cols) > 0:
    df_train = df_train.drop(columns=uniq_cols)

Dropping 0 columns with unique values


## Data scaling

In [13]:
# Normalize the data
scaler = MinMaxScaler()
# Scale only numerical columns
to_scale = list(set(df_train.select_dtypes(exclude=['object', 'category']).columns) - set(ad_label))
print("Scaling {} columns".format(len(to_scale)))
df_train[to_scale] = scaler.fit_transform(df_train[to_scale].values.astype(np.float64))
df_train.describe()

Scaling 107 columns


,isFraud,TransactionAmt,card1,card2,card3,card5,C1,C2,C3,C4,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
count,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,...,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,0.034990,0.004220,0.511539,0.525111,0.406068,0.724663,0.003008,0.002683,0.000217,0.001816,...,0.000711,0.004432,0.005761,0.005565,0.001172,0.001848,0.001647,0.000177,0.000404,0.000272
std,0.183755,0.007489,0.281741,0.313190,0.086423,0.299967,0.028510,0.027178,0.005790,0.030559,...,0.003122,0.019886,0.023063,0.024230,0.024217,0.029697,0.028365,0.003193,0.004550,0.003671
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.001349,0.288515,0.230000,0.381679,0.481752,0.000213,0.000176,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.002145,0.498850,0.522000,0.381679,0.919708,0.000213,0.000176,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.003906,0.757875,0.824000,0.381679,0.919708,0.000640,0.000527,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Categorical data

In [14]:
# One-hot encode categorical values
# Before, we save the original column names (will be used when applying the same preprocessing to the test set)
train_cols = set(df_train.columns) - set(ad_label)
cat_cols = list(set(df_train.select_dtypes(include=['object', 'category']).columns) - set(ad_label))
print("Converting categorical attributes {} to one-hot encoding".format(', '.join(cat_cols)))
one_hot = pd.get_dummies(df_train[cat_cols])
df_train = df_train.drop(columns=cat_cols)
df_train = df_train.join(one_hot)
df_train.describe()

Converting categorical attributes ProductCD to one-hot encoding


,isFraud,TransactionAmt,card1,card2,card3,card5,C1,C2,C3,C4,...,V317,V318,V319,V320,V321,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W
count,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,...,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,0.034990,0.004220,0.511539,0.525111,0.406068,0.724663,0.003008,0.002683,0.000217,0.001816,...,0.001848,0.001647,0.000177,0.000404,0.000272,0.116028,0.055922,0.063838,0.019690,0.744522
std,0.183755,0.007489,0.281741,0.313190,0.086423,0.299967,0.028510,0.027178,0.005790,0.030559,...,0.029697,0.028365,0.003193,0.004550,0.003671,0.320258,0.229771,0.244465,0.138934,0.436130
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.001349,0.288515,0.230000,0.381679,0.481752,0.000213,0.000176,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.002145,0.498850,0.522000,0.381679,0.919708,0.000213,0.000176,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.003906,0.757875,0.824000,0.381679,0.919708,0.000640,0.000527,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
df_train.head()

,isFraud,TransactionAmt,card1,card2,card3,card5,C1,C2,C3,C4,...,V317,V318,V319,V320,V321,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0.0,0.002137,0.743044,0.525111,0.381679,0.306569,0.000213,0.000176,0.0,0.0,...,0.000873,0.000000,0.0,0.0,0.0,0,0,0,0,1
2987001,0.0,0.000900,0.100885,0.608000,0.381679,0.014599,0.000213,0.000176,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0,0,0,0,1
2987002,0.0,0.001840,0.210566,0.780000,0.381679,0.481752,0.000213,0.000176,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0,0,0,0,1
2987003,0.0,0.001558,0.984824,0.934000,0.381679,0.124088,0.000427,0.000879,0.0,0.0,...,0.010476,0.008022,0.0,0.0,0.0,0,0,0,0,1
2987004,0.0,0.001558,0.201023,0.828000,0.381679,0.014599,0.000213,0.000176,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0,1,0,0,0


## SMOTE oversampling
Our data has a large class imbalance which can lead to overfitting on the majority class

In [16]:
smote = SMOTE()
df_train, train_y = smote.fit_resample(df_train.drop(columns='isFraud'), df_train['isFraud'])
df_train = pd.concat((df_train, train_y), axis=1)

In [17]:
# Finally, save the training set
df_train.to_csv(root + "/processed/clean/train_transaction_clean.csv", index=False)
df_train.head()

,TransactionAmt,card1,card2,card3,card5,C1,C2,C3,C4,C5,...,V318,V319,V320,V321,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,isFraud
0,0.002137,0.743044,0.525111,0.381679,0.306569,0.000213,0.000176,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0,0,0,0,1,0.0
1,0.000900,0.100885,0.608000,0.381679,0.014599,0.000213,0.000176,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0,0,0,0,1,0.0
2,0.001840,0.210566,0.780000,0.381679,0.481752,0.000213,0.000176,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0,0,0,0,1,0.0
3,0.001558,0.984824,0.934000,0.381679,0.124088,0.000427,0.000879,0.0,0.0,0.0,...,0.008022,0.0,0.0,0.0,0,0,0,0,1,0.0
4,0.001558,0.201023,0.828000,0.381679,0.014599,0.000213,0.000176,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0,1,0,0,0,0.0


## Test set preprocessing
The steps are the same as for the training set but they can be completed faster

In [26]:
# Repeat the preceding steps for the test set
test_trans_df = pd.read_csv(root + "/ieee-fraud-detection/test_transaction.csv", index_col='TransactionID')
test_id_df = pd.read_csv(root + "/ieee-fraud-detection/test_identity.csv", index_col='TransactionID')
df_test = test_trans_df.merge(test_id_df, how='left', left_index=True, right_index=True)
df_test.to_csv(root + "/processed/merged/test_set.csv", index=True)
del test_trans_df
del test_id_df

# Keep only the columns available during training
df_test = df_test[train_cols]
df_test.head()

# First, deal with missing values
cols_missing_vals = df_test.columns[df_test.isna().sum() >= 1]
df_test[cols_missing_vals].isna().sum().sort_values(ascending=False)
num_nan_cols = df_test[cols_missing_vals].select_dtypes(exclude=['object', 'category']).columns
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df_test[num_nan_cols] = imputer.fit_transform(df_test[num_nan_cols].values.astype(np.float64))

# Process numerical values
num_cols = df_test.select_dtypes(exclude=['object', 'category']).columns
scaler = MinMaxScaler()
# Scale only numerical columns
print("Scaling {} columns".format(len(num_cols)))
df_test[num_cols] = scaler.fit_transform(df_test[num_cols].values.astype(np.float64))

# Finally, process categorical values
cat_cols = df_test.select_dtypes(include=['object', 'category']).columns
one_hot = pd.get_dummies(df_test[cat_cols])
df_test = df_test.drop(columns=cat_cols)
df_test = df_test.join(one_hot)

# Finally, save the dataframe
df_test.to_csv(root + "/processed/clean/test_transaction_clean.csv")
df_test.isna().any().sum()

Scaling 106 columns


0

## Validate the preprocessing steps

In [37]:
df_train = pd.read_csv("C:/Users/verdi/Documents/Datasets/IEEE-CIS_Fraud_Detection/processed/clean/train_transaction_clean.csv")
df_test = pd.read_csv("C:/Users/verdi/Documents/Datasets/IEEE-CIS_Fraud_Detection/processed/clean/test_transaction_clean.csv", index_col="TransactionID")
submission = pd.DataFrame(columns=['TransactionID'], data=df_test.index)

assert (df_test.isna().any().sum() == 0 and df_train.isna().any().sum() == 0)

## Training a simple supervised model to test the processing

In [38]:
X_train = df_train.drop(columns=["isFraud"], axis=1).to_numpy()
X_test = df_test.to_numpy()
y_train = df_train["isFraud"].to_numpy()
del df_train
del df_test
X_train.shape[0] == y_train.shape[0] and X_train.shape[1] == X_test.shape[1]

True

In [31]:
model = CatBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)

Learning rate set to 0.208062
0:	learn: 0.5704404	total: 303ms	remaining: 5m 2s
1:	learn: 0.4869556	total: 421ms	remaining: 3m 30s
2:	learn: 0.4473522	total: 548ms	remaining: 3m 2s
3:	learn: 0.4015736	total: 683ms	remaining: 2m 49s
4:	learn: 0.3780410	total: 808ms	remaining: 2m 40s
5:	learn: 0.3539398	total: 952ms	remaining: 2m 37s
6:	learn: 0.3366261	total: 1.09s	remaining: 2m 35s
7:	learn: 0.3171441	total: 1.22s	remaining: 2m 31s
8:	learn: 0.3038625	total: 1.35s	remaining: 2m 29s
9:	learn: 0.2966965	total: 1.47s	remaining: 2m 25s
10:	learn: 0.2866670	total: 1.58s	remaining: 2m 21s
11:	learn: 0.2711913	total: 1.72s	remaining: 2m 22s
12:	learn: 0.2673570	total: 1.83s	remaining: 2m 19s
13:	learn: 0.2550893	total: 1.96s	remaining: 2m 17s
14:	learn: 0.2503645	total: 2.07s	remaining: 2m 15s
15:	learn: 0.2463506	total: 2.2s	remaining: 2m 15s
16:	learn: 0.2410873	total: 2.33s	remaining: 2m 15s
17:	learn: 0.2369392	total: 2.47s	remaining: 2m 14s
18:	learn: 0.2338993	total: 2.58s	remaining: 2m

In [44]:
submission = pd.concat([submission, pd.DataFrame(y_pred[:, 1], columns=['isFraud'])], axis=1)
submission.to_csv(root + "/submissions/catboost_ieee_fraud_detection.csv", index=False)